# GRPO Llama3.1-8B-Instruct Demo: Group Relative Policy Optimization

This tutorial demonstrates training the Llama3.1 8B-Instruct model on the GSM8K math reasoning benchmark using Group Relative Policy Optimization (GRPO). GRPO can enhance your model's problem-solving skills on mathematical word problems, coding problems, etc.

## What is GRPO?

GRPO is an RL algorithm designed to enhance the reasoning abilities of LLMs. It is a variant of Proximal Policy Optimization (PPO) that reduces memory usage by eliminating the need for a separate value function model. GRPO works by:

1. Generating multiple responses for a given prompt
2. Evaluating these responses using a reward model
3. Calculating a relative advantage based on the group's performance to update the policy

## Libraries Used

- **Tunix**: Library for GRPO implementation
- **vLLM**: Library for efficient model inference and generation
- **MaxText**: For model creation and training infrastructure

## Hardware Requirements

This tutorial uses a single host TPUVM such as `v6e-8/v5p-8`.


## Install Necessary Libraries

First, let's install the required dependencies:


In [ ]:
### (Optional) Run this if you just have this file and nothing else

# 1. Clone the MaxText repository (from AI‑Hypercomputer)
!git clone https://github.com/AI-Hypercomputer/maxtext.git

# 2. Navigate into the cloned directory
%cd maxtext

In [ ]:
### (Optional) Do not run this if you already installed the dependencies

# 3. Ensure setup.sh is executable
!chmod +x setup.sh

# 4. Execute the setup script
!./setup.sh

# Install vllm requirements
!./src/MaxText/examples/install_tunix_vllm_requirement.sh

# force numpy version
!pip install --force-reinstall numpy==2.1.2
# install nest_asyncio
!pip install nest_asyncio

import nest_asyncio

nest_asyncio.apply()
# To fix "This event loop is already running" error in Colab

## Imports

Import all necessary libraries for GRPO training:


In [ ]:
# Set up the variables for the script
import os
import sys

# Set the MaxText home directory (where you cloned the maxtext repo)
MAXTEXT_REPO_ROOT = os.path.expanduser("~") + "/maxtext"
print(f"MaxText Home directory (from Python): {MAXTEXT_REPO_ROOT}")

DEBUG = False  # set to True to run in debug mode, for more print statements
# set this to the path of the checkpoint you want to load, gs://<bucket> supported
MODEL_CHECKPOINT_PATH = "path/to/scanned/checkpoint"

In [ ]:
import functools
from pprint import pprint
import re
from etils import epath
from flax import linen as nn
from flax import nnx
from flax.linen import partitioning as nn_partitioning
import grain
import humanize
import jax
import numpy as np
import optax
from orbax import checkpoint as ocp
import tensorflow_datasets as tfds
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from tunix.models.llama3 import model as llama3_lib
from tunix.rl import rl_cluster as rl_cluster_lib
from tunix.rl.grpo.grpo_learner import GRPOConfig, GrpoLearner
from tunix.rl.rollout import base_rollout
from tunix.rl.rollout.base_rollout import RolloutConfig
from tunix.sft import metrics_logger

In [ ]:
from pathlib import Path
from typing import Any, Dict, Optional

maxtext_path = Path(f"{MAXTEXT_REPO_ROOT}") / "src" / "MaxText"

# Change working directory to MaxText project root
os.chdir(maxtext_path)
sys.path.insert(0, str(maxtext_path))

from MaxText import model_creation_utils, pyconfig
from MaxText.integration.tunix.tunix_adapter import TunixMaxTextAdapter

print(f"✓ Changed working directory to: {os.getcwd()}")
print(f"✓ MaxText project root: {maxtext_path}")
print(f"✓ Added to Python path: {maxtext_path}")

if not jax.distributed.is_initialized():
  jax.distributed.initialize()
jax.devices()

In [ ]:
# Hugging Face Authentication Setup
import os
from huggingface_hub import login

# Use your Hugging Face token (recommended)
# Get your token from: https://huggingface.co/settings/tokens

os.environ["HF_TOKEN"] = "hf_your_token_here"
login(token=os.environ["HF_TOKEN"])

## Hyperparameters

Let's define the configuration we are going to use. Note that this is by no means a "perfect" set of hyperparameters. To get good results, you might have to train the model for longer.

### Data Configuration


In [ ]:
# ====== Data ======
TRAIN_DATA_DIR = f"{MAXTEXT_REPO_ROOT}/data/train"
TEST_DATA_DIR = f"{MAXTEXT_REPO_ROOT}/data/test"
if not os.path.exists(TRAIN_DATA_DIR):
  os.makedirs(TRAIN_DATA_DIR)
if not os.path.exists(TEST_DATA_DIR):
  os.makedirs(TEST_DATA_DIR)
TRAIN_FRACTION = 1.0

### Checkpoint and Logging Configuration


In [ ]:
# ====== Checkpoint directory =====
LOG_DIR = f"{MAXTEXT_REPO_ROOT}/content/tensorboard/grpo/logs_llama3/"
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)

# ===== Profiling =====
PROFILE_DIR = f"{MAXTEXT_REPO_ROOT}/content/jax_traces/grpo/profiles_llama3/"
if not os.path.exists(PROFILE_DIR):
  os.makedirs(PROFILE_DIR)

# ====== Checkpoint saving ======
CKPT_DIR = f"{MAXTEXT_REPO_ROOT}/content/ckpts_llama3/"

if not os.path.exists(CKPT_DIR):
  os.makedirs(CKPT_DIR)

SAVE_INTERVAL_STEPS = 500
MAX_TO_KEEP = 4

# ====== Reproducibility ======
SEED = 42

### GRPO Configuration

GRPO-specific hyperparameters for generation and training:


In [ ]:
# ====== GRPO ======
# === Generation during GRPO training ===
MAX_PROMPT_LENGTH = 256
TOTAL_GENERATION_STEPS = 768
# Important to keep a high-ish temperature for varied, diverse responses during
# training.
TEMPERATURE = 0.9
TOP_P = 1.0
TOP_K = 50
# The number of times the policy generates multiple responses for a given prompt
# within a single training step. This corresponds to `G` in Algorithm 1 in the
# paper. The "group" in GRPO comes from here.
NUM_GENERATIONS = 2

# === other GRPO configs ===
# The number of iterations per batch (𝜇 in GRPO algo 1).
NUM_ITERATIONS = 1
# The coefficient for the KL divergence penalty (𝛽) in the GRPO loss function.
# Important to keep a high enough value for this, otherwise, the KL divergence
# can increase unchecked.
BETA = 0.08
# Epsilon value for clipping (𝜀 in GRPO loss in paper). Similar to PPO, for
# stable updates.
EPSILON = 0.2

### Training Configuration

Training hyperparameters including batch size, learning rate, and optimization settings:


In [ ]:
# ====== Training ======
BATCH_SIZE = 1
# Increase `NUM_BATCHES` and `MAX_STEPS` for better results.
NUM_BATCHES = 4  # 200
# Keep `NUM_TEST_BATCHES` low so that evaluation runs quickly. It can be
# increased to a max. of 330 (if batch size is 4).
NUM_TEST_BATCHES = 5  # 200

SEQUENCE_LENGTH = 1024

EVAL_EVERY_N_STEPS = 10  # this doesn't matter if `TRAIN_FRACTION = 1.0`.
NUM_EPOCHS = 1  # can potentially train for more epochs

# Number of training steps.
MAX_STEPS = int(NUM_BATCHES * NUM_ITERATIONS * TRAIN_FRACTION * NUM_EPOCHS)

# === AdamW, warmup, cosine scheduler ===
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1
# == Cosine decay with warmup scheduler ==
# Linearly increase learning rate from 0. to 5e-6 in the first 10% training
# steps, and then gradually decrease the learning rate to 0 using cosine
# scheduler.
WARMUP_STEPS = int(0.1 * MAX_STEPS)
# == Grad clipping ==
# Grad clipping to prevent large gradients. Found this
# important to keep KL divergence in check.
MAX_GRAD_NORM = 0.1

### Inference and Reward Configuration

Configuration for model inference and reward function parameters:


In [ ]:
# ====== Inference ======
GENERATION_CONFIGS = {
    # greedy search
    "greedy": {"temperature": 0.01, "top_k": 1, "top_p": 1.0},
    # some randomness
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    # liberal
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}

# ====== Reward ======
REWARD_EXACT_FORMAT_MATCH = 3.0
REWARD_WHITE_SPACE_FORMAT_MATCH = 1.5
REWARD_PARTIAL_FORMAT_MATCH = 0.5
REWARD_RATIO_GUESS_TO_ANSWER_HIGH = 0.5
REWARD_RATIO_GUESS_TO_ANSWER_LOW = 0.25
PENALTY_INCORRECT_FORMAT = -0.5
PENALTY_INCORRECT_ANSWER = -1.0

## Utility Functions

Helper functions for monitoring memory usage and other utilities:


In [ ]:
def show_hbm_usage():
  """Displays memory usage per device."""
  fmt_size = functools.partial(humanize.naturalsize, binary=True)

  for d in jax.local_devices():
    stats = d.memory_stats()
    used = stats["bytes_in_use"]
    limit = stats["bytes_limit"]
    print(f"Using {fmt_size(used)} / {fmt_size(limit)} ({used/limit:%}) on {d}")

## Data Preprocessing

First, let's define some special tokens. We instruct the model to first reason between the `<reasoning>` and `</reasoning>` tokens. After reasoning, we expect it to provide the answer between the `<answer>` and `</answer>` tokens.

We use OpenAI's GSM8K dataset. GSM8K comprises grade school math word problems.


In [ ]:
model_tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct"
)


reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"


SYSTEM_PROMPT = f"""You are given a problem. Think about the problem and \
provide your reasoning. Place it between {reasoning_start} and \
{reasoning_end}. Then, provide the final answer (i.e., just one numerical \
value) between {solution_start} and {solution_end}."""

TEMPLATE = """<start_of_turn>user
{system_prompt}

{question}<end_of_turn>
<start_of_turn>model"""

In [ ]:
def extract_hash_answer(text: str) -> str | None:
  if DEBUG:
    print(f"Extracting answer from: {text}")
  if "####" not in text:
    return None
  return text.split("####")[1].strip()


def get_dataset(data_dir, split="train") -> grain.MapDataset:
  # Download data
  if not os.path.exists(data_dir):
    os.makedirs(data_dir)

  data = tfds.data_source(
      "gsm8k",
      split=split,
      data_dir=data_dir,
      builder_kwargs={"file_format": tfds.core.FileFormat.ARRAY_RECORD},
      download=True,
  )

  loaded_dataset = (
      grain.MapDataset.source(data)
      .shuffle(seed=SEED)
      .map(
          lambda x: {
              # passed to model forward pass
              "prompts": model_tokenizer.apply_chat_template(
                  [
                      {
                          "role": "user",
                          "content": TEMPLATE.format(
                              system_prompt=SYSTEM_PROMPT,
                              question=x["question"].decode("utf-8"),
                          ),
                      },
                  ],
                  tokenize=False,
                  add_generation_prompt=True,
              ),
              # passed to reward functions
              "question": x["question"].decode("utf-8"),
              # passed to reward functions
              "answer": extract_hash_answer(x["answer"].decode("utf-8")),
          }
      )
  )
  return loaded_dataset

In [ ]:
dataset = get_dataset(TRAIN_DATA_DIR, "train").batch(BATCH_SIZE)[:NUM_BATCHES]

if TRAIN_FRACTION == 1.0:
  train_dataset = dataset.repeat(NUM_EPOCHS)
  val_dataset = None
else:
  train_dataset = dataset[: int(len(dataset) * TRAIN_FRACTION)]
  train_dataset = train_dataset.repeat(NUM_EPOCHS)

  val_dataset = dataset[int(len(dataset) * TRAIN_FRACTION) :].repeat(NUM_EPOCHS)

test_dataset = get_dataset(TEST_DATA_DIR, "test").batch(BATCH_SIZE)[
    :NUM_TEST_BATCHES
]


# Let's see how one batch of the dataset looks like!


if DEBUG:
  for ele in train_dataset[:1]:
    pprint(ele)

## Load the Policy Model and the Reference Model

The policy model is the model which is actually trained and whose weights are updated. The reference model is the model with which we compute KL divergence. This is to ensure that the policy updates are not huge and that it does not deviate too much from the reference model.

Typically, the reference model is the base model, and the policy model is the same base model, but with potentially LoRA parameters where only the LoRA parameters are updated. This script is not using LoRA, so both the reference and policy models are the same.

Note: We perform full precision (fp32) training. You can, however, leverage Qwix for QAT.


In [ ]:
print("HBM usage before loading model:")
show_hbm_usage()

### Load MaxText Model


In [ ]:
def get_ref_maxtext_model(config):

  model, mesh = model_creation_utils.create_nnx_model(config)
  with mesh:
    tunix_model = TunixMaxTextAdapter(
        base_model=model,
    )

    model_config = llama3_lib.ModelConfig.llama3_1_8b()
    tunix_model.config = model_config

  return tunix_model, mesh


model_config = llama3_lib.ModelConfig.llama3_1_8b()

### Load Reference Model

Note: pass the path to your scanned checkpoint for "load_parameters_path". To create a scanned checkpoint, you can use `/maxtext/src/MaxText/utils/ckpt_conversion/to_maxtext.py`


In [ ]:
# Load the reference model
config_ref = pyconfig.initialize(
    [
        "",
        f"{MAXTEXT_REPO_ROOT}/src/MaxText/configs/base.yml",
    ],
    base_output_directory="dummy",  # This is not used in Tunix.
    run_name="test-tunix-maxtext-llama3.1-8b",
    tokenizer_type="tiktoken",
    tokenizer_path="assets/tokenizer_llama3.tiktoken",
    load_parameters_path=f"{MODEL_CHECKPOINT_PATH}",
    max_target_length=SEQUENCE_LENGTH,
    async_checkpointing="false",
    model_name="llama3.1-8b",
    skip_jax_distributed_system="true",
    weight_dtype="bfloat16",
    attention="dot_product",
    remat_policy="custom",
    decoder_layer_input="offload",
    query_proj="offload",
    key_proj="offload",
    value_proj="offload",
)

llama3_1_8b, mesh = get_ref_maxtext_model(config_ref)

llama3_1_8b.config = model_config

nnx.display(llama3_1_8b)

In [ ]:
if DEBUG:
  print("Model initialized successfully")
  print(f"Model mesh shape: {mesh.shape}")
  print(f"Model config: {model_config}")

  # Sanity check that weights are loaded correctly
  _maxtext_state_flatten = nnx.state(llama3_1_8b).flat_state()
  maxtext_state_flatten = {
      ".".join(str(key) for key in keys): v
      for keys, v in _maxtext_state_flatten
  }
  print(
      f"maxtext_state_flatten[base.token_embedder.embedding].value={maxtext_state_flatten['base.token_embedder.embedding'].value}"
  )


# See the memory use after loading the reference model:
print("HBM usage after loading ref model:")
show_hbm_usage()

### Load Policy Model

Note: pass the path to your scanned checkpoint for "load_parameters_path". To create a scanned checkpoint, you can use `/maxtext/src/MaxText/utils/ckpt_conversion/to_maxtext.py`

TODO: @mazumdera: change this to use lora


In [ ]:
config_policy = pyconfig.initialize(
    [
        "",
        f"{MAXTEXT_REPO_ROOT}/src/MaxText/configs/base.yml",
    ],
    base_output_directory="dummy",  # This is not used in Tunix.
    run_name="test-tunix-maxtext-llama3.1-8b",  # This is not used in Tunix.
    tokenizer_type="tiktoken",
    tokenizer_path="assets/tokenizer_llama3.tiktoken",
    load_parameters_path=f"{MODEL_CHECKPOINT_PATH}",
    max_target_length=SEQUENCE_LENGTH,
    async_checkpointing="false",
    model_name="llama3.1-8b",
    skip_jax_distributed_system="true",
    weight_dtype="bfloat16",
    attention="dot_product",
    remat_policy="custom",
    decoder_layer_input="offload",
    query_proj="offload",
    key_proj="offload",
    value_proj="offload",
)
llama3_1_8b_policy, mesh_policy = get_ref_maxtext_model(config_policy)

llama3_1_8b_policy.config = model_config

nnx.display(llama3_1_8b_policy)

In [ ]:
if DEBUG:
  print("Model initialized successfully")
  print(f"Model mesh shape: {mesh_policy.shape}")
  print(f"Model config: {model_config}")

  # Sanity check that weights are loaded correctly
  _maxtext_state_flatten = nnx.state(llama3_1_8b_policy).flat_state()
  maxtext_state_flatten = {
      ".".join(str(key) for key in keys): v
      for keys, v in _maxtext_state_flatten
  }
  print(
      f"maxtext_state_flatten[base.token_embedder.embedding].value={maxtext_state_flatten['base.token_embedder.embedding'].value}"
  )

# See memory usage after loading the policy model:
print("HBM usage after loading policy model:")
show_hbm_usage()

## Define Reward Functions

We define four reward functions:

1. **Format Matching**: Reward if the format of the output exactly matches the instruction given in `TEMPLATE`
2. **Approximate Format Matching**: Reward if the format of the output approximately matches the instruction given in `TEMPLATE`
3. **Answer Correctness**: Reward if the answer is correct/partially correct
4. **Number Extraction**: Sometimes, the text between `<answer>`, `</answer>` might not be one number. So, extract the number, and reward the model if the answer is correct.

The reward functions are inspired from [here](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb).

First off, let's define a RegEx for checking whether the format matches.


In [ ]:
match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{reasoning_start}.+?{reasoning_end}.*?"
    rf"{solution_start}(.+?){solution_end}"
    rf"[\s]{{0,}}$",
    flags=re.MULTILINE | re.DOTALL,
)

match_format.search(
    f"{reasoning_start}Let me"
    f" think!{reasoning_end}{solution_start}2{solution_end}",
)

### Reward Function 1: Exact Format Matching

Give the model a reward of 3 points if the format matches exactly.


In [ ]:
def match_format_exactly(prompts, completions, **kargs):
  scores = []
  for completion in completions:
    score = 0
    response = completion
    # Match if format is seen exactly!
    if match_format.search(response) is not None:
      score += REWARD_EXACT_FORMAT_MATCH
    scores.append(score)
  return scores

### Reward Function 2: Approximate Format Matching

We also reward the model if the format of the output matches partially.


In [ ]:
def match_format_approximately(prompts, completions, **kargs):
  scores = []

  for completion in completions:
    score = 0
    response = completion
    # Count how many keywords are seen - we penalize if too many!
    # If we see 1, then plus some points!
    score += (
        REWARD_PARTIAL_FORMAT_MATCH
        if response.count(reasoning_start) == 1
        else PENALTY_INCORRECT_FORMAT
    )
    score += (
        REWARD_PARTIAL_FORMAT_MATCH
        if response.count(reasoning_end) == 1
        else PENALTY_INCORRECT_FORMAT
    )
    score += (
        REWARD_PARTIAL_FORMAT_MATCH
        if response.count(solution_start) == 1
        else PENALTY_INCORRECT_FORMAT
    )
    score += (
        REWARD_PARTIAL_FORMAT_MATCH
        if response.count(solution_end) == 1
        else PENALTY_INCORRECT_FORMAT
    )
    scores.append(score)
  return scores

### Reward Function 3: Answer Correctness

Reward the model if the answer is correct. A reward is also given if the answer does not match exactly, i.e., based on how close the answer is to the correct value.


In [ ]:
def check_answer(prompts, completions, answer, **kargs):
  responses = completions

  extracted_responses = [
      guess.group(1) if (guess := match_format.search(r)) is not None else None
      for r in responses
  ]

  scores = []
  for guess, true_answer in zip(extracted_responses, answer):
    score = 0
    if guess is None:
      scores.append(0)
      continue
    # Correct answer gets 3 points!
    if guess == true_answer:
      score += REWARD_EXACT_FORMAT_MATCH
    # Match if spaces are seen
    elif guess.strip() == true_answer.strip():
      score += REWARD_WHITE_SPACE_FORMAT_MATCH
    else:
      # We also reward it if the answer is close via ratios!
      # Ie if the answer is within some range, reward it!
      try:
        ratio = float(guess) / float(true_answer)
        if ratio >= 0.9 and ratio <= 1.1:
          score += REWARD_RATIO_GUESS_TO_ANSWER_HIGH
        elif ratio >= 0.8 and ratio <= 1.2:
          score += REWARD_RATIO_GUESS_TO_ANSWER_LOW
        else:
          score += PENALTY_INCORRECT_ANSWER  # Penalize wrong answers
      except:
        score += PENALTY_INCORRECT_FORMAT  # Penalize
    scores.append(score)
  return scores

### Reward Function 4: Number Extraction

Sometimes, the text between `<answer>` and `</answer>` might not be one number; it can be a sentence. So, we extract the number and compare the answer.


In [ ]:
match_numbers = re.compile(
    rf"{solution_start}.*?([\d\.]{{1,}})", flags=re.MULTILINE | re.DOTALL
)
match_numbers.findall(f"{solution_start}  0.34  {solution_end}")


def check_numbers(prompts, completions, answer, **kargs):
  question = kargs["question"]
  responses = completions

  extracted_responses = [
      guess.group(1) if (guess := match_numbers.search(r)) is not None else None
      for r in responses
  ]

  scores = []
  if DEBUG:
    print("START ============================")
    print(f"Question: {question[0]}")
    print(f"Answer: {answer[0]}")
    print(f"Response: {responses[0]}")
    print(f"Extracted: {extracted_responses[0]}")
    print("END ==============================")
  for guess, true_answer in zip(extracted_responses, answer):
    if guess is None:
      scores.append(0)
      continue
    # Convert to numbers
    try:
      true_answer = float(true_answer.strip())
      guess = float(guess.strip())
      scores.append(1.5 if guess == true_answer else 0.0)
    except:
      scores.append(0)
      continue
  return scores

## Evaluation Functions

Before we train the model, let's evaluate the model on the test set so we can see the improvement post training.

We evaluate it in two ways:

**Quantitative**
- **Answer Accuracy**: percentage of samples for which the model predicts the correct final numerical answer
- **Answer (Partial) Accuracy**: percentage of samples for which the model predicts a final numerical answer such that the `model answer / answer` ratio lies between 0.9 and 1.1.
- **Format Accuracy**: percentage of samples for which the model outputs the correct format, i.e., reasoning between the reasoning special tokens, and the final answer between the `<start_answer>`, `<end_answer>` tokens.

**Qualitative**

We'll also print outputs for a few given questions so that we can compare the generated output later.


In [ ]:
def generate_responses(
    prompts,
    rl_cluster,
    num_passes=1,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
):
  """Generate responses for a batch of prompts across multiple passes.

  Args:
      prompts: List of prompts to generate responses for
      rl_cluster: Model cluster for generation
      num_passes: Number of generation passes
      temperature: Sampling temperature
      top_k: Top-k sampling parameter
      top_p: Top-p sampling parameter

  Returns:
      List of lists containing responses for each prompt across passes
  """
  multiple_call_responses = [[] for _ in range(len(prompts))]

  for p in range(num_passes):
    responses = rl_cluster.rollout.generate(
        prompts,
        rollout_config=RolloutConfig(
            max_tokens_to_generate=TOTAL_GENERATION_STEPS,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        ),
    )
    responses = responses.text

    if DEBUG:
      print(f"Pass {p+1}/{num_passes}, responses: {responses}")

    for idx, response in enumerate(responses):
      multiple_call_responses[idx].append(response)

  return multiple_call_responses

In [ ]:
def score_responses(question, responses, answer):
  """Score a set of responses for a single question.

  Args:
      question: The evaluation question
      responses: List of generated responses for this question
      answer: The correct answer

  Returns:
      Tuple of (is_correct, is_partially_correct, has_correct_format)
  """
  if DEBUG:
    print("========================================")
    print(f"Evaluation Question: {question}")
    print(f"Evaluation Answer: {answer}")
    print(f"Evaluation Responses: {responses}")
    print("========================================")

  is_correct = False
  is_partially_correct = False
  has_correct_format = False

  for response in responses:
    # Extract numerical response
    extracted_response = (
        guess.group(1)
        if (guess := match_numbers.search(response)) is not None
        else "-1000000"
    )

    if DEBUG:
      print(f"Evaluation extracted_response: {extracted_response}")

    # Check exact correctness
    try:
      if float(extracted_response.strip()) == float(answer.strip()):
        is_correct = True

      # Check partial correctness (within 10%)
      ratio = float(extracted_response.strip()) / float(answer.strip())
      if 0.9 <= ratio <= 1.1:
        is_partially_correct = True
    except Exception as e:
      if DEBUG:
        print(f"Evaluation Exception: {e}")
        print("SKIPPED")

    # Check format correctness
    if match_format.search(response) is not None:
      has_correct_format = True

    # Early exit if all criteria are met
    if is_correct and is_partially_correct and has_correct_format:
      break

  return is_correct, is_partially_correct, has_correct_format

In [ ]:
def evaluate(
    dataset,
    rl_cluster,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_passes=1,
    corr_lst=False,
    make_lst=False,
):
  """Computes accuracy and percentage of outputs matching the format.

  Args:
      dataset: The evaluation dataset
      rl_cluster: Model cluster for generation
      temperature: Sampling temperature
      top_k: Top-k sampling parameter
      top_p: Top-p sampling parameter
      num_passes: Number of generation passes
      corr_lst: If True, only include correct responses in the list
      make_lst: If True, return a list of (question, answer, responses)

  Returns:
      Tuple of statistics and optionally the response list
  """
  response_lst = []
  corr = 0
  partially_corr = 0
  corr_format = 0
  total = 0

  for batch in tqdm(dataset):
    answers = batch["answer"]
    questions = batch["question"]
    prompts = batch["prompts"]

    # Generate responses for all prompts in the batch
    multiple_call_responses = generate_responses(
        prompts=prompts,
        rl_cluster=rl_cluster,
        num_passes=num_passes,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )

    # Score each question-answer pair
    for question, responses, answer in zip(
        questions, multiple_call_responses, answers
    ):
      is_correct, is_partially_correct, has_correct_format = score_responses(
          question=question,
          responses=responses,
          answer=answer,
      )

      # Update counters
      if is_correct:
        corr += 1
        if corr_lst and make_lst:
          response_lst.append((question, answer, responses))
      else:
        if not corr_lst and make_lst:
          response_lst.append((question, answer, responses))

      if is_partially_correct:
        partially_corr += 1

      if has_correct_format:
        corr_format += 1

      total += 1

      # Print progress every 10 items
      if total % 10 == 0:
        print(
            f"===> {corr=}, {total=}, {corr / total * 100=}, "
            f"{partially_corr / total * 100=}, {corr_format / total * 100=}"
        )

  # Prepare return values
  to_return = (
      corr,
      total,
      corr / total * 100,
      partially_corr / total * 100,
      corr_format / total * 100,
  )

  if make_lst:
    return to_return, response_lst
  return to_return

## Training Setup

Let's set up all the configs first - checkpointing, metric logging and training. We then train the model.


In [ ]:
# Ckpt saving
checkpointing_options = ocp.CheckpointManagerOptions(
    save_interval_steps=SAVE_INTERVAL_STEPS, max_to_keep=MAX_TO_KEEP
)

# Metrics logger
metrics_logging_options = metrics_logger.MetricsLoggerOptions(
    log_dir=LOG_DIR, flush_every_n_steps=20
)


# Logs
print(f"TensorBoard logs directory: {LOG_DIR}")
print(f"tensorboard --logdir {LOG_DIR} --port=8086")

In [ ]:
# Optimizer, learning rate scheduler, gradient clipping
optimizer = optax.adamw(
    learning_rate=optax.schedules.warmup_cosine_decay_schedule(
        init_value=0.0,
        peak_value=LEARNING_RATE,
        warmup_steps=WARMUP_STEPS,
        decay_steps=MAX_STEPS,
        end_value=0.0,
    ),
    b1=B1,
    b2=B2,
    weight_decay=WEIGHT_DECAY,
)

if MAX_GRAD_NORM is not None:
  optimizer = optax.chain(
      optax.clip_by_global_norm(max_norm=MAX_GRAD_NORM),
      optimizer,
  )

In [ ]:
# RL Cluster config
# Note that we use vLLM as the rollout engine.
# and we are using Tensor Parallelism for rollout

cluster_config = rl_cluster_lib.ClusterConfig(
    role_to_mesh={
        rl_cluster_lib.Role.ACTOR: mesh,
        rl_cluster_lib.Role.REFERENCE: mesh,
        rl_cluster_lib.Role.ROLLOUT: mesh,
    },
    rollout_engine="vllm",
    offload_to_cpu=False,
    training_config=rl_cluster_lib.RLTrainingConfig(
        actor_optimizer=optimizer,
        eval_every_n_steps=EVAL_EVERY_N_STEPS,
        max_steps=MAX_STEPS,
        gradient_accumulation_steps=1,
        # metrics logging
        metrics_logging_options=metrics_logging_options,
        # checkpoint saving
        checkpoint_root_directory=CKPT_DIR,
        checkpointing_options=checkpointing_options,
    ),
    rollout_config=base_rollout.RolloutConfig(
        max_tokens_to_generate=TOTAL_GENERATION_STEPS,
        max_prompt_length=MAX_PROMPT_LENGTH,
        kv_cache_size=MAX_PROMPT_LENGTH + TOTAL_GENERATION_STEPS + 256,
        temperature=TEMPERATURE,
        top_p=TOP_P,
        top_k=TOP_K,
    ),
    rollout_vllm_model_version="meta-llama/Meta-Llama-3.1-8B-Instruct",
    rollout_vllm_hbm_utilization=0.2,
    rollout_vllm_tpu_backend_type="jax",
)

grpo_config = GRPOConfig(
    num_generations=NUM_GENERATIONS,
    num_iterations=NUM_ITERATIONS,
    beta=BETA,
    epsilon=EPSILON,
)

In [ ]:
# RL cluster

rl_cluster = rl_cluster_lib.RLCluster(
    actor=llama3_1_8b_policy,
    reference=llama3_1_8b,
    tokenizer=model_tokenizer,
    cluster_config=cluster_config,
)

# GRPO Trainer
grpo_trainer = GrpoLearner(
    rl_cluster=rl_cluster,
    reward_fns=[
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    grpo_config=grpo_config,
)

In [ ]:
if DEBUG:
  # verify if vllm sampler works
  output = rl_cluster.rollout.generate(
      ["The capital of France is"],
      rollout_config=RolloutConfig(max_tokens_to_generate=64, temperature=0.1),
  )

  print(f"Output: {output}")

## Evaluate Before Training


In [ ]:
(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset,
    rl_cluster,
    **GENERATION_CONFIGS["greedy"],
)
print(
    f"Pre GRPO Training: {corr=}, {total=}, {accuracy=}%, {partial_accuracy=}%,"
    f" {format_accuracy=}%"
)

## Start Training


In [ ]:
jax.profiler.start_trace(PROFILE_DIR)
with mesh, nn_partitioning.axis_rules(config_policy.logical_axis_rules):
  grpo_trainer.train(dataset)
jax.profiler.stop_trace()

print("HBM usage after training:")
show_hbm_usage()

## Final Evaluation

Let's evaluate our model after training!


In [ ]:
(corr, total, accuracy, partial_accuracy, format_accuracy) = evaluate(
    test_dataset,
    rl_cluster,
    **GENERATION_CONFIGS["greedy"],
)
print(
    f"Post GRPO Training: {corr=}, {total=}, {accuracy=}%,"
    f" {partial_accuracy=}%, {format_accuracy=}%"
)